# <span style="color:#F72585"><center>Optimización con torch </center> </span>


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Pytorch/Imagenes/640px-Ackley.gif" width="600" height="400" align="center" /> 
</center>   
</figure>
<center>

Fuente: <a href="https://commons.wikimedia.org/wiki/File:Ackley.gif">Pablormier</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA 4.0</a>, via Wikimedia Commons

</center>


## <span style="color:#4361EE">Introducción</span>

Ahora que tenemos un modelo y datos, es hora de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. 

Entrenar un modelo es un proceso iterativo; en cada iteración (llamada época) el modelo hace una suposición sobre la salida, calcula el error en su suposición (pérdida), recopila las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior) y optimiza estos parámetros usando el descenso de gradiente.

## <span style="color:#4361EE">Implementación completa</span>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# datos
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# modelo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)


# hiperparámetros
learning_rate = 1e-3
batch_size = 64
epochs = 5

# loop de entrenamiento

# función de pérdida
loss_fn = nn.CrossEntropyLoss()

# Optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# función de entrenamiento
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # calcula predicción y pérdida
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()# por defecto el calculo del gradiente es acumulativo
        loss.backward() # clacula y acumula los gradientes
        optimizer.step()# paso de optimización x += grad
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

# función de validación
def test_loop(testloader, model, loss_fn):
    size = len(testloader.dataset)
    num_batches = len(testloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in testloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# ciclo de entrenamiento
epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n -------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')


Epoch 1
 -------------------------
loss: 2.298553 [    0/60000]
loss: 2.298919 [ 6400/60000]
loss: 2.283872 [12800/60000]
loss: 2.282191 [19200/60000]
loss: 2.275929 [25600/60000]
loss: 2.249293 [32000/60000]
loss: 2.267802 [38400/60000]
loss: 2.242098 [44800/60000]
loss: 2.258423 [51200/60000]
loss: 2.213442 [57600/60000]
Test Error: 
 Accuracy: 31.6%, Avg loss: 2.230083 

Epoch 2
 -------------------------
loss: 2.244263 [    0/60000]
loss: 2.255225 [ 6400/60000]
loss: 2.208153 [12800/60000]
loss: 2.216929 [19200/60000]
loss: 2.186004 [25600/60000]
loss: 2.144744 [32000/60000]
loss: 2.192587 [38400/60000]
loss: 2.138104 [44800/60000]
loss: 2.186815 [51200/60000]
loss: 2.095315 [57600/60000]
Test Error: 
 Accuracy: 33.4%, Avg loss: 2.127904 

Epoch 3
 -------------------------
loss: 2.164541 [    0/60000]
loss: 2.183559 [ 6400/60000]
loss: 2.092442 [12800/60000]
loss: 2.112276 [19200/60000]
loss: 2.037690 [25600/60000]
loss: 1.989364 [32000/60000]
loss: 2.059463 [38400/60000]
loss: 1.

## <span style="color:#4361EE">Referencias</span> 

1. Basado en los [tutoriales de Pytorch](https://pytorch.org/tutorials/)
1. [Deep learning for coders with FastAI and Pytorch](http://library.lol/main/F13E85845AE48D9FD7488FE7630A9FD3)